<a href="https://colab.research.google.com/github/carlaGuzman511/PlantsDiseasePredictiveModel/blob/main/Train_Plant_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Plant Disease Prediction**

Dataset: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

Importar las libraries

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Preprocesamiento de los datos

1. set de imagenes de entrenamiento

https://keras.io/api/data_loading/image/

documentation: https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

NotFoundError: Could not find directory train

Preprocesamiento de las imagenes

Set de imagenes de validacion

documentation: https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory[texto del enlace](https://)

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)


**Crear el Modelo**

In [ ]:
cnn = tf.keras.models.Sequential()

**Crear las capas convoluciones**

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:

cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))


In [ ]:
cnn.add(tf.keras.layers.Dropout(0.25))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1500,activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dropout(0.4)) #Para evitar el sobreajuste

In [ ]:
# Capa de salida
cnn.add(tf.keras.layers.Dense(units=38,activation='softmax'))

**Fase de compilación y entrenamiento**

In [ ]:
cnn.compile(optimizer=tf.keras.optimizers.legacy.Adam(
    learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
cnn.summary()

***Para evitar la función de pérdida excesiva***

1. Elija una tasa de aprendizaje pequeña por defecto 0.001 aquí hemos tomado 0.0001
2. Puede haber posibilidades de un ajuste insuficiente, **por lo que debe aumentar el número de neuronas.**
3. **Agregue más capas convolucionales para extraer más características de las imágenes.** Es posible que el modelo no pueda capturar características relevantes o que el modelo sea confuso debido a la falta de características, por lo que debe alimentar con más características.

In [ ]:
training_history = cnn.fit(x=training_set,validation_data=validation_set,epochs=10)

**Evaluacion del modelo**

In [ ]:
#precisio del set de entrenamiento
train_loss, train_acc = cnn.evaluate(training_set)
print('training loss: ', train_loss)
print('Training accuracy:', train_acc)

In [ ]:
# precision del set de validacion
val_loss, val_acc = cnn.evaluate(validation_set)
print('validation loss: ', val_loss)
print('Validation accuracy:', val_acc)

**guardamos en modelo**

In [ ]:
cnn.save('trained_plant_disease_model.keras')

In [ ]:
training_history.history #retornamos un dictionario con el historial

In [ ]:
#guardamos el historial en formato json
import json
with open('training_hist.json','w') as f:
  json.dump(training_history.history,f)

In [ ]:
print(training_history.history.keys())

**visualizacion de la precision del modelo**

In [ ]:
#epocas
epochs = [i for i in range(1,11)]

plt.plot(epochs,training_history.history['accuracy'],color='red',label='Precision del entrenamiento')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Precision de la validacion')

plt.xlabel('Nro. de epocas')
plt.title('Visualizacion de los resultados de precision')
plt.legend()

plt.show()

**otras metricas para la evaluacion del modelo**

In [ ]:
class_name = validation_set.class_names

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
y_pred = cnn.predict(test_set)
predicted_categories = tf.argmax(y_pred, axis=1)

In [ ]:
true_categories = tf.concat([y for x, y in test_set], axis=0)
Y_true = tf.argmax(true_categories, axis=1)

In [ ]:
Y_true

In [ ]:
predicted_categories

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(Y_true,predicted_categories)

In [ ]:
# Puntuación F de recuperación de precisión
print(classification_report(Y_true,predicted_categories,target_names=class_name))

**Visualización de matriz de confusión**

In [ ]:
plt.figure(figsize=(40, 40))
sns.heatmap(cm,annot=True,annot_kws={"size": 10})

plt.xlabel('Predicted Class',fontsize = 20)
plt.ylabel('Actual Class',fontsize = 20)

plt.title('Plant Disease Prediction Confusion Matrix',fontsize = 25)
plt.show()